In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('cluster').getOrCreate()

In [3]:
df = spark.read.csv('FileStore/tables/seeds_dataset.csv', header=True, inferSchema=True)

In [4]:
df.printSchema()

root
-- area: double (nullable = true)
-- perimeter: double (nullable = true)
-- compactness: double (nullable = true)
-- length_of_kernel: double (nullable = true)
-- width_of_kernel: double (nullable = true)
-- asymmetry_coefficient: double (nullable = true)
-- length_of_groove: double (nullable = true)

In [5]:
from pyspark.ml.clustering import KMeans

In [6]:
from pyspark.ml.feature import VectorAssembler

In [7]:
assembler = VectorAssembler(inputCols=df.columns,
                            outputCol='features')

In [8]:
final_data = assembler.transform(df)

In [9]:
final_data.printSchema()

root
-- area: double (nullable = true)
-- perimeter: double (nullable = true)
-- compactness: double (nullable = true)
-- length_of_kernel: double (nullable = true)
-- width_of_kernel: double (nullable = true)
-- asymmetry_coefficient: double (nullable = true)
-- length_of_groove: double (nullable = true)
-- features: vector (nullable = true)

In [10]:
from pyspark.ml.feature import StandardScaler

In [11]:
scaler = StandardScaler(inputCol='features', outputCol='scaled_features')

In [12]:
scaled_data = scaler.fit(final_data).transform(final_data)

In [13]:
scaled_data.head()

Out[ 13 ]: Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22, features=DenseVector([15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22]), scaled_features=DenseVector([5.2445, 11.3633, 36.8608, 13.0072, 8.7685, 1.4772, 10.621]))

In [14]:
kmeans = KMeans(featuresCol='scaled_features', k=3)

In [15]:
model = kmeans.fit(scaled_data)

In [16]:
wsse = model.computeCost(scaled_data)
wsse

Out[ 16 ]: 429.07559671506715

In [17]:
centers = model.clusterCenters()
centers

Out[ 17 ]: 
[array([ 6.31670546, 12.37109759, 37.39491396, 13.91155062,
 9.748067 , 2.39849968, 12.2661748 ]),
 array([ 4.87257659, 10.88120146, 37.27692543, 12.3410157 ,
 8.55443412, 1.81649011, 10.32998598]),
 array([ 4.06105916, 10.13979506, 35.80536984, 11.82133095,
 7.50395937, 3.27184732, 10.42126018])]

In [18]:
model.transform(scaled_data).select('prediction').show()

+----------+
prediction|
+----------+
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 0|
 0|
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 1|
 2|
+----------+
only showing top 20 rows